In [ ]:
!pip install transformers
!pip install datasets
!pip install arabert
!pip install sentencepiece
!pip install  arabicnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 28.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186

In [ ]:
import torch
from torch import nn


In [ ]:
import re
import string

def _remove_diacritics(text):
    text = re.sub(r'\s*[A-Za-z]+\b', '' , text)
    return  re.sub(r"[ًًٌٍَُِّْ]", "", text)

def _remove_extra_spaces(text):
    return re.sub(" +", " ", text)

def _add_spaces_to_all_special_chars(text):
    return re.sub(r"(?<=\w)([؟.,،])", r" \1", text)

def _remove_repeated_chars(text):
    return re.sub(r"(.)\1+", r"\1\1", text)

def _remove_qoutes(text):
    #text = re.sub(r'"(.*?)"',"",text) # remove the text between two double qoutes
    #text = re.sub("https?:\/\/.*[\r\n]*", "", text) # remove the urls
    text = re.sub(r"[0-9]"," ",text)
    return re.sub(r"\[(.*?)\]", " " , text) # remove the text between two brackets
def _remove_puncs(text):
    puncs = string.punctuation.replace(",","").replace("،","").replace("?","؟")
    puncs = puncs.replace(")","").replace("(","")
    text = text.translate(str.maketrans(' ', ' ', puncs))
    text = text.replace("•","")
    text = text.replace(")"," ").replace("("," ").replace(":","")
    return text
def Sequential(fns):
    def new_fn(inputs):
      for fn in fns:
        inputs = fn(inputs)
      return inputs
    return new_fn
WS = ['نشاط إثرائي اقرأ ثم استنتج','عرف ذلك من النص التالي','السخلة : هي ولد الفتم من الضأن والماعز ساعة وضعه',
      'مصر من الفتح الإسلامي حتى قيام الدول المستقلة',
      'مصر في عصر الولاة','اقرأ','لتتعرف','التالي','ذلك','النص',
      'لاحظ الخريطة التالية لتتعرفها','فكر وناقش']

WS2 = ['مرحلة التاسيس و الاستقرار',
       'كل من',
       'اقرأ الشكل التالي لتتعرف أشهرهم بشيء من التفصيل',
       'بشيء من التفصيل',
       'حمورابي بالأكدية تلفظ امورابي وتعني المعتلي',
       'والآن اقرأ الشكل التالي لتتعرف أدوار الجهاد ضد الصليبيين',
       '(بدر الجمالي)','هل تعرف لمـاذا','لاحظ الخريطة الزمنية التالية لتتعرفها',
       'ومن خلال','الخريطة الزمنية السابقة'
       ,'نستنتج أن',
       'رضي الله عنه',
       'نعم يا أحبابنا','ق . م',
       'التواصل الثقافي والفني مع أفريقيا',
       'الصـور المقابلة',
       'مـن خـلال',
       'الدرس الثاني',
       'الدرس الثالث',
       'الدرس الرابع',
       'الدرس الخامس',
       'الدرس',
       'ولنبدأ',
       'والان تعال',
       'والآن تعال معا',
       'استثمر المصرى',
       'من خلال ما يلي','والآن تعال معنا',"أولا","رابعا","خامسا","سادسا","سابعا","والآن لاحظ الشكل التالي"]

WORDS=['هل','اقرأ وناقش','معلومة إثرائية','علام يدل','للصف الثاني الثانوي','أ-','ب-','ج-','وهذا ما سوف نتعرفه في الدرس القادم','عزيزي الطالب / عزيزتي الطالبة',
         'اقرأ النص التالي','ولعلك تتساءل عزيزي الطالب','يمكنك الإجابة عن هذا التساؤل بعد قراءة النص التالي','وذلك ما سوف نتعرفه في الدرس القادم',
  'هل تعرف أسباب فشلها  اقرأ الشكل التالي لتتعرف أهمها','تعرف ذلك من النص التالي',"لاحظ الخريطة التالية",
          "وذلك ما سوف نتعرفه في الدرس القادم","والآن تعال معا",
       'الفتوحات الأموية في الغرب الإسلامي',"ثالثا","ثانيا",
       "اولا","اقرأ وناقش،المعاني والقيم الواردة",
       "الفتوحات الإسلامية في عصر الدولة الأموية",
       'لاحظ الخريطة التالية لتتعرف عليها',
       "بعد ان","وبعد ان القينا الضوء علي",
       'وهذا ما سوف نتعرفه',
       'الوحدة التالية',
       'رواه مسلم','العامل التاريخي',
       'لتتعرفها','النص التالي',"أ.","ب.","ج.","د.","والآن لاحـظ الشك","ثانيا:",".. الرسم والنقش:","سادسا:"]
print(len(WORDS))
WORDS += WS
WORDS += WS2
WORDS = list(set(WORDS))
text_processor = Sequential([ _remove_diacritics,
                             _remove_qoutes,
                              _remove_puncs,
                              _remove_extra_spaces,
                             _remove_repeated_chars ]) #223



42


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
len(WORDS)

87

In [ ]:


from arabert.preprocess import ArabertPreprocessor
preprocessor = ArabertPreprocessor(model_name="")

In [ ]:
def freeze(model):
    for p in model.parameters():
        p.requires_grad = False
    return model

In [ ]:
from transformers import BertTokenizer,AutoModelForSeq2SeqLM, pipeline,AutoModel,AutoTokenizer,BartForConditionalGeneration


model_mini= "asafaya/bert-mini-arabic"
model_med = "asafaya/bert-medium-arabic"
model_bart = "moussaKam/AraBART"
model_bart_2 = "abdalrahmanshahrour/auto-arabic-summarization"
model_name="malmarjeh/mbert2mbert-arabic-text-summarization"

model = AutoModelForSeq2SeqLM.from_pretrained(model_bart_2)
tokenizer = AutoTokenizer.from_pretrained(model_bart_2)

#encoder = freeze(AutoModel.from_pretrained(model_bart_2).encoder)
print(f"num of parameters is :{sum([p.numel() for p in model.parameters()])}")

num of parameters is :139221504


In [ ]:
import pandas as pd
vals = pd.read_json('/content/drive/MyDrive/data/labeled_validation_dataset.jsonl',lines=True)
vals['document'] = vals['paragraph']
del vals['paragraph']
vals.head()

,example_id,summary,document
0,0,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...
1,1,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...
2,2,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...
3,3,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...
4,4,الشعب السوري يصر على استقلال بلدهم من السيطرة ...,السوريون يصرون على استقلال بلادهم : و مثلما رف...


In [ ]:
get_len = lambda x: len(tokenizer.tokenize(x))
test_data = pd.read_json('/content/drive/MyDrive/data/validation_data.jsonl',lines=True)
test_data['document'] = test_data['paragraph']
del test_data['paragraph']
test_data['document_len'] = test_data['document'].apply(get_len)
test_data['document_len'].max()

778

In [ ]:
vals['document_len'] = vals['document'].apply(get_len)
vals['summary_len'] = vals['summary'].apply(get_len)
vals['document_len'].min()

274

In [ ]:
pred = pd.read_json('predictions.jsonl',lines=True)

In [ ]:
test_data['summary'] = pred['summary']
test_data['summary_len'] = test_data['summary'].apply(get_len)
test_data.head()

,example_id,document,document_len,summary,summary_len
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...,460,فتح بيت المقدس 15 ه 636 م لحق عمرو بن العاص با...,290
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...,173,بعد أن انتهى عمر بن الخطاب من خطبته دعاه البطر...,124
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...,125,قام المسلمون بالتوجه غربا لفتح شمال إفريقيا عا...,50
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...,268,ألقينا الضوء على أهم فتوحات بلاد شمال إفريقيا ...,87
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...,116,توقفت حركة الفتوحات الإسلامية بسبب فتنة عثمان ...,44


In [ ]:
exts  = pd.read_json('/content/exts.jsonl',lines=True)
exts.head()

,document,summary,example_id
0,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...,توقفت حركه الفوتوحات بعدفتنه عثمان حتي قيام ال...,4.0
1,: الفتوحات الأموية في المشرق الإسلامي.\nحرص ول...,حرص ولاه بني اميه علي استكمال فوتوحات الراشدون...,5.0
2,الفتوحات الأموية في آسيا الصغرى (بلاد الروم).\...,الفوتوحات الامويه في اسيا الصغري امتدت حتي ارم...,6.0
3,الفتوحات الإسلامية في عصر الدولة الأموية .\nوو...,واصل الامويون فتوحاتهم حتي نجحوا في اخضاع اق...,7.0
4,حسان بن النعمان: والذي انطلق إلى إفريقية في عه...,حسان بن النعمان والذي انطلق إلى إفريقية في ع...,8.0


In [ ]:
exts['document_len'] = exts['document'].apply(get_len)
exts['summary_len'] = exts['summary'].apply(get_len)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/first_9.csv')
data2 = pd.read_csv("/content/drive/MyDrive/data/all_data_25_45_words.csv")
del data['address']
del data2['Unnamed: 0']
data = pd.concat([data,data2]).drop_duplicates().reset_index(drop=True)
data['document'] = data['article']
del data['article']
len(data)

209933

In [ ]:
data['sum_len'] = data['summary'].apply(lambda x:len(x.split()))
data['para_len']= data['document'].apply(lambda x:len(x.split()))

In [ ]:
data['calc'] = data['sum_len']/data['para_len']
data = data[(data['calc']<0.40)& (data['calc']>0.3)].reset_index(drop=True).drop_duplicates()
len(data)

63604

In [ ]:
vals['sum_len'] = vals['summary'].apply(lambda x:len(x.split()))
vals['para_len']= vals['document'].apply(lambda x:len(x.split()))
test_data['sum_len'] = test_data['summary'].apply(lambda x:len(x.split()))
test_data['para_len']= test_data['document'].apply(lambda x:len(x.split()))

vals['calc'] = vals['sum_len']/vals['para_len']
test_data['calc'] = test_data['sum_len']/test_data['para_len']

#vals = vals[(vals['calc']<0.41)& (vals['calc']>0.3)].reset_index(drop=True)
#test_data = test_data[(test_data['calc']<0.41)& (test_data['calc']>0.3)].reset_index(drop=True)

In [ ]:
test_data = test_data[['document','summary']]
vals = vals[['document','summary']]
data = data[['document','summary']]
data = pd.concat([data,vals]).reset_index(drop=True).drop_duplicates()
len(data)

63717

In [ ]:
from torch.utils.data import Dataset,DataLoader


class SummarizationDataset(Dataset):
  def __init__(self,
               data,
               with_summary=True):
      self.data = data
      self.with_summary = with_summary
  def __len__(self):
      return len(self.data)
  def __getitem__(self,idx):
     document = self.data.iloc[idx]['document']
    # for word in WORDS :
     #   document = document.replace(word,'')
     document = text_processor(document)
     document = _remove_extra_spaces(document.replace("\n"," "))
     if self.with_summary:
        summary = self.data.iloc[idx]['summary']
        return document,summary
     return document

def collate_fn(batch):
    inputs,labels = [],[]
    with_summary =  len(batch[0])>1
    for item in batch:
       inputs.append(item[0])
       if with_summary:
        labels.append(item[1])

    inputs = tokenizer(inputs,
                       return_tensors='pt',
                       padding='longest',
                       truncation=True,
                       max_length=500)
    if with_summary:
      labels =  tokenizer(labels,
                       return_tensors='pt',
                       padding='longest',
                       truncation=True,
                       max_length=250)
      dec_in = {k:v[:,:-1] for k,v in labels.items()}
      dec_out = {k:v[:, 1:] for k,v in labels.items()}
      dec_in = {'decoder_input_ids':dec_in['input_ids'],
                "decoder_attention_mask":dec_in['attention_mask']}
      inputs.update(dec_in)
      return inputs,dec_out
    return inputs


In [ ]:

train_ds = SummarizationDataset(data)
val_ds = SummarizationDataset(test_data)

train_iter = DataLoader(train_ds,batch_size=18,
                        shuffle=True,
                        num_workers=2,
                        pin_memory=True,
                        collate_fn=collate_fn)

val_iter = DataLoader(val_ds,
                      batch_size=4,
                      num_workers=2,
                      pin_memory=True,
                      collate_fn=collate_fn)


In [ ]:
next(iter(val_iter))

[{'input_ids': tensor([[    0,   525,    11,  ..., 39841,    31,     2],
         [    0, 16105,    22,  ...,     1,     1,     1],
         [    0, 37316,    22,  ...,     1,     1,     1],
         [    0,   525,    11,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]), 'decoder_input_ids': tensor([[    0,   678,   790,  3935,   379,   333,   243,  6440,    71, 10174,
           1797,    67, 10691,    61,  5765,  3881,   578,  4821,   241,  4603,
          10016,     6,  3872,     9, 36800,   589, 32329,    45,    19,   131,
            320, 15246,  4465, 29144,     5,  2870, 37612,  5136,   552,     4,
          12392,  4821,    99, 11752,  5607,  1839,   117,  3815,     5, 13887,
             26,   552,  1798,  2267, 29580,   785, 11033,  1011,  5759,     4,
          24544,    11,   172, 13887,    26,  7459,   440,    67,  2321,  4439,
         

In [ ]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = tokenizer.vocab_size

In [ ]:
loss_obj = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
def accuracy(preds,labels,attn_mask):
    dim = preds.shape[-1]
    pred = preds.view(-1,dim)
    label = labels.reshape(-1)
    attn= attn_mask.reshape(-1)
    pred = pred.argmax(dim=-1)
    acc = (pred==label)*attn
    return acc.sum()/attn.sum()

def greedy_decode_loss(model,context,targets):
    bs, seq_len = targets.size()
    dec_in = torch.ones([bs,1],dtype=torch.int32).to(targets.device)*tokenizer.cls_token_id
    context = model.model.encoder(**context).last_hidden_state
    loss = 0
    for i in range(1,seq_len):
        logits = model.model.decoder(input_ids=dec_in,encoder_hidden_states=context).last_hidden_state
        logits = model.lm_head(logits)
        tar = targets[:,i]
        pred = logits[:,-1,:]
        l = loss_obj(pred,tar)
        dec_in = torch.concat([dec_in,pred.argmax(dim=-1,keepdim=True)],dim=-1)
        loss = loss + l
        del logits, pred
    loss = loss/(seq_len-1)
    hidden_tar = (encoder(input_ids=targets).last_hidden_state).mean(dim=-1)
    hidden_pred = (encoder(input_ids=dec_in).last_hidden_state).mean(dim=-1)
    sim_loss = - torch.nn.functional.cosine_similarity(hidden_tar,hidden_pred).mean()
    return loss, sim_loss


def loss_fn(logits,targets):
    return loss_obj(logits.reshape(-1,logits.size(-1))
                     ,targets.reshape(-1))

In [ ]:
from torch.cuda.amp import GradScaler


mean = lambda x:sum(x)/len(x)
DEVICE = torch.device("cuda")
scaler = GradScaler()


def train_epoch(model,train_iter,opt,times=5):
    losses,accs = [],[]
    model.train()
    for i,(enc_in,labels) in enumerate(train_iter):
      enc_in = {k:v.to(DEVICE) for k,v in enc_in.items()}
      labels = {k:v.to(DEVICE) for k,v in labels.items()}
      opt.zero_grad()
      with torch.autocast(device_type='cuda', dtype=torch.float16):
        out = model(**enc_in).logits
        loss = loss_fn(out,labels['input_ids'])
      #scaled_grad_params = torch.autograd.grad(outputs=scaler.scale(loss),
      #                                           inputs=model.parameters(),
      #                                           create_graph=True)

      #inv_scale = 1./scaler.get_scale()
      #grad_params = [p * inv_scale for p in scaled_grad_params]

        # Computes the penalty term and adds it to the loss
      #with torch.autocast(device_type='cuda', dtype=torch.float16):
       #     grad_norm = 0
        #    for grad in grad_params:
         #       grad_norm += grad.pow(2).sum()
          #  grad_norm = grad_norm.sqrt()
           # loss = loss + grad_norm

      scaler.scale(loss).backward()
      scaler.step(opt)
      scaler.update()
      loss = loss.item()
      acc = accuracy(out,
                     labels['input_ids'],
                     labels['attention_mask']).item()
      del out, enc_in, labels
      torch.cuda.empty_cache()
      if (i+1)%(len(train_iter)//times)==0:
        print(f"Finished Training on {(i+1)*100/len(train_iter):.2f} % of the data, loss:{loss:.3f}, acc:{acc:.3f}.")
      losses.append(loss)
      accs.append(acc)
    return mean(losses),mean(accs)



@torch.no_grad()
def val_epoch(model,val_iter):
    losses,accs = [],[]
    model.eval()
    for enc_in,labels in val_iter:
        enc_in = {k:v.to(DEVICE) for k,v in enc_in.items()}
        labels = {k:v.to(DEVICE) for k,v in labels.items()}
        with torch.autocast(device_type='cuda', dtype=torch.float16):
             out = model(**enc_in).logits
             loss = loss_fn(out,labels['input_ids'])
        loss = loss.item()
        acc = accuracy(out,
                      labels['input_ids'],
                      labels['attention_mask']).item()
        losses.append(loss)
        accs.append(acc)
        del out,enc_in,labels
        torch.cuda.empty_cache()
    return mean(losses),mean(accs)

In [ ]:
import math
class LoraLayer(nn.Module):
  def __init__(self,orig_layer,
               rank=32,
               drop=0.1,
               alpha=32.0):
      super().__init__()
      assert rank > 0
      self.scale = alpha/rank
      weight_shape = orig_layer.weight.shape

      self.drop = nn.Dropout(drop) if drop !=0 else nn.Idenity()
      self.lora_A = nn.Parameter(torch.zeros(rank,weight_shape[1])) # (rank, in_shape)
      self.lora_B = nn.Parameter(torch.zeros(weight_shape[0],rank)) # (out_shape, rank)
      nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
      nn.init.zeros_(self.lora_B)
      self.orig_layer = orig_layer
      self.merged = False

  def reset_parameters(self):
      self.orig_layer.reset_parameters()
      nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
      nn.init.zeros_(self.lora_B)

  def merge(self):
      self.orig_layer.weight.data += (self.lora_B @ self.lora_A).transpose(0,1)*self.scale
      self.merged=True

  def forward(self,x):
      out =self.orig_layer(x)
      if not self.merged:
         out2 = self.drop(x) @ self.lora_A.transpose(0,1)
         out2 = (out2  @ self.lora_B.transpose(0,1))*self.scale
         return out2+out
      return out

# freeze the whole model except for lora parameters
def freeze_lora(model):
    for name,parameter in model.named_parameters():
        if "lora_" not in name:
            parameter.requires_grad=False
    return model
def merge_lora(ch):
    if not ch.merged:
       ch.merge()  # merge the weights if its not merged
    return ch.orig_layer


def get_apply(Module,
              instance,
              attr_names,
              map_fn,**map_fn_kwargs):
    for name, ch in Module.named_children():
      if isinstance(ch,LoraLayer):
           continue
      elif isinstance(ch,instance):
        for attr_name in attr_names:
          attr = map_fn(getattr(ch,attr_name),**map_fn_kwargs)
          setattr(ch,attr_name,attr)
      else:
           get_apply(ch,instance,attr_names,map_fn,**map_fn_kwargs)
def convert_lora(module,**kwargs):
    return LoraLayer(module,**kwargs)



In [ ]:
model = model.from_pretrained("/content/model_0.79")

In [ ]:
def get_params(model):
    trainable = sum([p.numel() for p in model.parameters() if p.requires_grad])
    untrainable = sum([p.numel() for p in model.parameters() if not p.requires_grad])
    return {"trainable":trainable,"untrainable":untrainable,"all":(trainable+untrainable)}
get_params(model)

{'trainable': 139221504, 'untrainable': 0, 'all': 139221504}

In [ ]:

model = model.cuda()
opt = torch.optim.AdamW(model.parameters(),lr=5e-5)

In [ ]:
import time
EPOCHS = 2
best_val = .355

for e in range(EPOCHS):
  print(f"Started Training on epoch:{e+1}/{EPOCHS}")
  st = time.time()
  train_loss, train_acc = train_epoch(model,train_iter,opt)
  val_loss, val_acc = val_epoch(model,val_iter)
  if val_acc>best_val:
    best_val = val_acc
    model.save_pretrained(f'model_{val_acc:.2f}')
  print(f"Finished Trainin in {(time.time()-st)/60:.2f} mins, train_loss:{train_loss:.3f}, train_acc:{train_acc:.3f},val_loss:{val_loss:.3f},val_acc:{val_acc:.3f}\n.")

Started Training on epoch:1/2
Finished Training on 20.00 % of the data, loss:2.850, acc:0.442.
Finished Training on 40.00 % of the data, loss:2.923, acc:0.451.
Finished Training on 60.00 % of the data, loss:3.041, acc:0.432.
Finished Training on 80.00 % of the data, loss:3.221, acc:0.405.
Finished Training on 100.00 % of the data, loss:3.039, acc:0.423.
Finished Trainin in 30.92 mins, train_loss:3.033, train_acc:0.430,val_loss:1.456,val_acc:0.767
.
Started Training on epoch:2/2
Finished Training on 20.00 % of the data, loss:2.912, acc:0.444.
Finished Training on 40.00 % of the data, loss:2.545, acc:0.506.


In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = model.from_pretrained("/content/model_0.79").cuda()

In [ ]:
!pip install rouge
from rouge import Rouge

In [ ]:
rouge = Rouge()


In [ ]:
def prepare_document(document,tok,max_len=256):
    text1 = [d for d  in document.split(".")if len(d)>5]
    text2 = [d for d in document.split('،') if len(d)>5]
    if len(text1) >len(text2):
      text = text1
      split_on = '.'
    else:
      text = text2
      split_on = '،'
    length = 0
    count = -1
    chunks = []
    chunk = ''
    if len(text)==1:
       text = [ d for d in document.split(",") if len(d)>5]
       split_on = '،'
    else: text
    text = re.split("والتي|وتم|وكانت|والذي|ومن|وعلي|فقد|أيضا|فتم|فمن|فاصبحت|وهي|و هي|و من|و مع|وهذا|وفي",document) if (len(text)<2) else text
    for sent in text:
      com_length = len(sent.split()) +length
      count += 1
      if com_length< max_len:
        chunk = split_on.join([chunk,sent])
        length = com_length
        if count == len(text)-1:
          chunks.append(chunk.strip()[1:])
      else:
        chunks.append(chunk.strip()[1:])
        chunk = ''
        chunk = split_on.join([chunk,sent])
        length = len(tokenizer.tokenize(sent))
    chunks = [c for c in chunks if len(c)>5]
    return chunks,split_on


@torch.no_grad()
def generate(model,tok,text,**kwargs):
    inputs = tok(text,return_tensors='pt',truncation=True)
    if kwargs is not None:
       num_returns = kwargs.get("num_return_sequences",None)
    inputs = {k:v.cuda() for k,v in inputs.items()}
    gen = model.generate(**inputs,**kwargs)
    gen = gen.cpu().squeeze()
    if num_returns is not None:
      return tok.batch_decode(gen,skip_spectial_tokens=True)
    return tok.decode(gen,skip_special_tokens=True)

test_index = 1
text = list(vals['document'])[test_index]
text = text_processor(text)
text = _remove_extra_spaces(text.replace("\n"," "))
texts,split = prepare_document(text,tokenizer,max_len=150)
print(len(texts),len(text.split()))
sums = []
for text in texts:
    text = preprocessor.preprocess(text)
    summary = generate(model,tokenizer,text,max_length=100,
             num_beams=10,
             no_repeat_ngram_size=2,
             repetition_penalty=2.0,
             length_penalty=1.0,
             )

    sums.append(summary)

summary = (split+" ").join([s.replace('.',"") for s in sums])
rouge.get_scores(summary,list(vals['summary'])[test_index])[0]['rouge-l']['f']

2 256


0.1718749951477052

In [ ]:
model.save_pretrained("/content/drive/MyDrive/data/model_.363")

In [ ]:
summary

'كانت منطقة الساحل الشمالي لسوريا ، تابعة للإدارة المركزية للمالك الفنيقية والفينيقيون أنتجوا الأصبغة الملونة بالحضارة الإنسانية ، وتعتبر مهنتهم الأساسية تقوم على التجارة البحرية بين حوض البحر المتوسط وخلال القرن التاسع قبل الميلاد، سقطت المدينة في يد الآشوريين وأخيرا الفرس بدخول الإسكندر المقدوني إلى سوريا في القرن السابع قبل الميلاد وفي عام 476 ق م دخل المدينة منفتخة في الحقيقة على الثقافة اليونانية التي كانت سائدة في الداخل السوري ، ناشرا ثقافات الحضارات عبر التاريخ إلى أن فتحها اللاذقية تحت الحكم البابلي، تستعد مدينة اللاذقية لإحياء ذكرى مولد الإمبراطورية السريانية ، وذلك بعد عامًا من احتفالات بلادها بتعامد الشمس على وجه تمثال الملك رمسيس الثانى بالعاصمة اليونانية أثينا، أعاد بناء اللاذقية الساحلية سلوقس الأول نيكاتور ، مؤسس مدينة آوديكيا اليونانية ، بنائها عام قبل الميلاد ، حسبما ذكر موقع روسيا اليوم، تمر اليوم الذكرى الـ ، على اكتشاف مقبرة فرعونية تحت حكم سلوكس نيكاتور ، وهي مدينة صينية مبنية من القنوات المائية'

In [ ]:
text = list(vals['document'])[test_index]
text = preprocessor.preprocess(text)
text = text.replace("\n","")
s = generate(model,tokenizer,text,max_length=200,
             num_beams=10,
             no_repeat_ngram_size=2,
             repetition_penalty=2.0,
             length_penalty=1.0,
             )
rouge.get_scores(s,list(vals['summary'])[test_index])[0]['rouge-l']['f']

0.05769230375739672

In [ ]:
torch.cuda.empty_cache()


In [ ]:
sim_model = freeze(AutoModel.from_pretrained(model_bart_2).encoder.cuda())
sim_tok = AutoTokenizer.from_pretrained(model_bart_2)
def get_sim(text1,text2):
    text1 = {k:v.to(DEVICE) for k,v in sim_tok(text1,return_tensors='pt').items()}
    text2 = {k:v.to(DEVICE) for k,v  in sim_tok(text2,return_tensors='pt').items()}
    hidden1 = sim_model(**text1).last_hidden_state
    hidden2 = sim_model(**text2).last_hidden_state
    hidden1 = hidden1.mean(dim=1)
    hidden2 = hidden2.mean(dim=1)
    sim =torch.nn.functional.cosine_similarity(hidden1,hidden2)
    return sim.item()

Some weights of the model checkpoint at abdalrahmanshahrour/auto-arabic-summarization were not used when initializing MBartModel: ['final_logits_bias', 'lm_head.weight']
- This IS expected if you are initializing MBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def process(doc):
    for word in WORDS :
        doc = doc.replace(word,'')
    doc = text_processor(doc)
    doc = _remove_extra_spaces(doc.replace("\n"," "))
    return doc
get_sim(exts['summary'].iloc[0],process(exts['document'].iloc[0]))

0.8266966342926025

In [ ]:
test_df = pd.read_json('/content/drive/MyDrive/data/validation_data.jsonl',lines=True)

In [ ]:
from tqdm import tqdm
import numpy as np
gens = []
sim_scores =[]
rouge_scores = []
changed = []
lens = []
for i,doc in tqdm(enumerate(test_df['paragraph'])):
    #for word in WORDS :
      #  doc = doc.replace(word,'')
    doc = text_processor(doc)
    doc = _remove_extra_spaces(doc.replace("\n"," "))
    if len(tokenizer.tokenize(doc)) >100:
      sumss,scores = [], []
      ls = [128, 150, 50, 75]
      for l in  ls:
        texts, split = prepare_document(doc,tokenizer,max_len=l)
        sums = []
        for text in texts:
          # text = preprocessor.preprocess(text)
            summary = generate(model,tokenizer,text,
                              max_length=150,
                              num_beams=10,
                              no_repeat_ngram_size=2,
                              repetition_penalty=3.0,
                              length_penalty=1.0,)
            sums.append(summary)
        summary = (split+" ").join([s.replace('.',"") for s in sums])
        sumss.append(summary)
        scores.append(get_sim(summary,doc))

      summary = sumss[np.argmax(scores)]
      score = scores[np.argmax(scores)]
      best_len = ls[np.argmax(scores)]

      lens.append((score,best_len,len(doc.split())))
    else:
       texts, split = prepare_document(doc,tokenizer,max_len=55)
       sums = []
       for text in texts:
          # text = preprocessor.preprocess(text)
          summary = generate(model,tokenizer,text,
                            max_length=300,
                            num_beams=10,
                            no_repeat_ngram_size=3,
                            repetition_penalty=3.0,
                            length_penalty=1.0,)
          sums.append(summary)
       summary = (split+" ").join([s.replace('.',"") for s in sums])
       score = get_sim(summary,doc)
    #score = get_sim(summary,doc)
    rouge_scores.append(rouge.get_scores(summary,list(pred['summary'])[i])[0]['rouge-l']['f'])
    gt_score = get_sim(pred['summary'][i],doc)
    if gt_score > score:
       gens.append(pred['summary'][i])
       sim_scores.append(gt_score)
       changed.append(0)
    else:
       changed.append(1)
       gens.append(summary)
       sim_scores.append(score)



272it [21:18,  4.70s/it]


In [ ]:
scores = [f[0] for f in lens]
max_lens = [f[1] for f in lens]
seq_lens = [f[2] for f in lens]

In [ ]:
sum(changed)

8

In [ ]:
mean(sim_scores)

0.8912617947687121

In [ ]:
ss = {'scores':scores,"max_len":max_lens,"seq_len":seq_lens}

In [ ]:
df = pd.DataFrame(ss)
df.to_csv('max_length.csv')

In [ ]:
df.head()

,scores,max_len,seq_len
0,0.979193,150,337
1,0.963573,75,124
2,0.928714,75,86
3,0.904851,100,182
4,0.935105,64,84


In [ ]:
get_sim(process(test_df['paragraph'][0]),pred['summary'][0])

0.959555983543396

In [ ]:
print(f"rouge: {mean(rouge_scores):.2f}, cos_sim: {mean(sim_scores):.2f}")

rouge: 0.44, cos_sim: 0.89


In [ ]:
def min_k(array,sl):
    arr = np.sort(array)[sl]
    indices = [np.nonzero(array==i)[0] for i in arr]
    return arr, np.concatenate(indices)
min_k(np.array(sim_scores),slice(0,10))

(array([0.5566802 , 0.58024895, 0.62714118, 0.62781996, 0.6409626 ,
        0.64835519, 0.64837521, 0.66001654, 0.6612407 , 0.66906011]),
 array([ 61,  53,  75, 160,  49,  43, 153,  58,  47,  74]))

In [ ]:
doc = process(test_df['paragraph'].iloc[43])

summary =  generate(model,tokenizer,doc,
                                  max_length=300,
                                  num_beams=8,
                                  no_repeat_ngram_size=2,
                                  repetition_penalty=3.0,
                                  length_penalty=1.0)
get_sim(summary,doc)

0.5171349048614502

In [ ]:
pred['summary'][53]

'كانت الصناعة أقل نسبيا من الزراعة من حيث أهميتها، أنشأت روما مصانع صغيرة من أجل صناعة الفخار بجانب الصناعات اليدوية، كما تم استخراج كميات كبيرة من الذهب والفضة والتي تستخدم في صناعة العملة والمجوهرات، وفرت المحاجر الحجارة لبناء المشروعات المختلفة'

In [ ]:
sim_scores

0.9007388195570778

In [ ]:
t = np.array(gens)[np.array(changed,dtype=bool)].tolist()[-1]

In [ ]:
len(t.split())

252

In [ ]:
len(test_df['paragraph'][271].split())

383

In [ ]:
252/383

0.6579634464751958

In [ ]:
model =model.cuda()

In [ ]:
scss = []
for i,idx in tqdm(enumerate(range(len(test_df)))):
    doc = test_df['paragraph'][idx]
    #for word in WORDS :
     #       doc = doc.replace(word,'')
    doc = text_processor(doc)
    doc = _remove_extra_spaces(doc.replace("\n"," "))
    summary =  generate(model,tokenizer,doc,
                                  max_length=300,
                                  num_beams=8,
                                  no_repeat_ngram_size=2,
                                  repetition_penalty=3.0,
                                  length_penalty=1.0,do_sample=True)
    sc = get_sim(summary,doc)
    scss.append((sim_scores[i],sc))
    if sc > sim_scores[i]:
      sim_scores[i] = sc
      gens[i] = summary


272it [05:53,  1.30s/it]


In [ ]:
indices = [old<new for old,new in scss]
np.array(sim_scores)[indices]

array([0.9033426 , 0.86349201, 0.79642832, 0.87678075])

In [ ]:
np.array(sim_scores).mean()

0.9060768822536749

In [ ]:
text = test_data['document'][66]
for word in WORDS :
        text=text.replace(word,' ')
text = text_processor(text).replace("\n", "")
text

'الحياة الفنيةظلت الفنـون مزدهرة فـي مصـر، ويشهد بذلـك مـا عثـر عليـه فـي مـدينـة الإسكندرية ولا يزال معروضا حتى الآن بالمتحف اليوناني الروماني ويمكن تقسيم الفـن حينذاك إلى قسمين رئيسين فن الطبقة الحاكمة ويمثله تماثيل الأباطرة الرومان والنقوش التي زينت بـهـا جـدران التوابيت الحجريةالفن الشعبي الذي يتمثل في آلاف التماثيل الصغيرة والأواني الفخارية و الحجرية والمعدنية التي تزخر بـهـا المتاحف، ك كثر استخدام الفسيفساء في الأرضيات بوجـه خـاص، تمثـل مناظرها أشخاص وحيوانات منهـا الحقيقيـة ومنها الخياليةالفن القبطى وفي ظل المسيحية نشأ فـن يعبر عن الدين الجديـد فـي إطـار المـوروث العميـق مـن الفـن المصرى والشرقي وقـد عـرف هذا الفن بالفن القبطي، وتمثله في الكنائس والأديرة القديمة والتي لاتزال آثارها باقية حـتـى الـيـوم وتأثر الفنان القبطى بخصائص الفـن المصرى القديـم فـي التصوير والرسم التشكيلي والنحت والعمارة ومن أمثلة الآثار القبطية في مصر مايليالكنيسة المعلقة وتقع في حي مصر القديمة بمحافظة القاهرة'

In [ ]:
test_df = pd.read_json('/content/drive/MyDrive/data/validation_data.jsonl',lines=True)
test_df

,example_id,paragraph
0,0,وبعد أن ألقينا الضوء على أهم فتوحات بلاد الشام...
1,1,الفتوحات الإسلامية في عصر الخلفاء الراشدين .\n...
2,2,فتوحات بلاد شمال إفريقيا .\nقام المسلمون بالتو...
3,3,وبعد أن ألقينا الضوء على أهم فتوحات بلاد شمال ...
4,4,عزيزي الطالب / عزيزتي الطالبة ...\nبعد أن توقف...
...,...,...
267,267,وظهرت كثير من الدراسات قصرت اهتمامها على الأدو...
268,268,كانت الزراعة المصرية في هذه الحقبة تمر بعصرها ...
269,269,حشدت فرنسا قواتها وأعلنت الحرب في 19 يوليو أي ...
270,270,تعتبر قناة السويس هي من أهم الطرق الملاحية الت...


In [ ]:
test_df['summary'] = gens

In [ ]:
test_df.drop(['paragraph'],axis=1,inplace=True)


In [ ]:
test_df.to_json("predictions15.jsonl", lines=True, orient='records', force_ascii=False)

In [ ]:
model.save_pretrained("/content/drive/MyDrive/data/model")